# Todos

## General

- bale ta ballast trips apo ships tensor pisw sto ships df ❌
- ananewne ka8w fora ta contracts_df kai ships_df me oti allagh ginetai stous contracts_tensor kai ships_tensor


## Step

### Allakse to state otan allazei h mera ❌

afou looparw se ola ta available ships shmainei oti h mera allazei

   #### - diamorfwse katallhla to ships log ❌
   
          * to ships log prepei na exei values >= 0 ❌
          * apo mh mhdenika ships log values afairese 1 epeidh perase mia mera ❌
          * an prokypsoun ships_log[ship] = 0 tote kane to `ship_availability` autou tou ship 1 ❌
          * osa ships einai available (exoun ships_log[ship] == 0) bainoun se lista `available_ships` pou 8a thn allazeis ka8e mera. ❌
           

   #### -  kane generate 4 nea contracts ❌

### Pws ypologizw to `reward` apo to action  ✅
    

   #### - Value of contract ✅
   #### - CII ✅
   #### - Lateness ✅
   #### - Ftiakse sugentrwtiko `calculate_reward(selected_ship_idx,selected_contract,selected_speed)` function ✅

### Pws na kanw swsta update to `state`  ✅
  

      
   #### - contract tensor ✅
   #### - ship tensor ✅
   #### - contract mask ✅
   #### - ship mask ✅
   #### - ship_log ✅
        

## Train 

ti einai ta `advantages`, `returns` kai giati xrhsimopoioyntai:
        
- ❌ sthn baseline_net.update(observations, returns) gia to update to baselineNet
- ❌ sthn PolicyGradient.update_policy(observations, actions, advantages)gia to update ths policygradient 

# Code

In [1]:
# autoreloading modules that i change in vscode
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [ ]:
# load data
data_dict = {
    "fleet_path": "data/fleet_small.csv",
    "ports_path": "data/ports_10.csv",
    "dm_path": "data/distance_matrix.csv",
}


ports_df = pd.read_csv(data_dict["ports_path"])
fleet_df = pd.read_csv(data_dict["fleet_path"])
distance_matrix = pd.read_csv(data_dict["dm_path"])

In [13]:
from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel
from training.training_functions import PolicyGradient

In [14]:
envo = CarbonEnv()

In [15]:
model = PolicyGradient(envo)

In [16]:
model.train()

Ksekina to year: 0
Xronia: 0 kai mera: 0
Ta available ships einai [1, 2, 3, 4]
Xronia: 0 kai mera: 1
Ta available ships einai [1, 4]
Den phra contract gia to available ship 4 
Xronia: 0 kai mera: 2
Ta available ships einai [4]
Xronia: 0, sunolo apo steps: 7 
To synoliko reward gia to year 0 htan 11.565890789031982
The prediction is[433.45096]
The loss is 178614.734375
The prediction is[-208853.47]
The loss is 43619176448.0
The prediction is[93.12628]
The loss is 12413.7548828125
The prediction is[2876.6843]
The loss is 8070941.0
The prediction is[-8552.2]
The loss is 73659176.0
The prediction is[1272.8422]
The loss is 1586127.75
The prediction is[3194.955]
The loss is 10114835.0
Average reward for batch 0: 12.16
Ksekina to year: 1
Xronia: 1 kai mera: 0
Ta available ships einai [1, 2, 3, 4]
Xronia: 1 kai mera: 1
Ta available ships einai [4]
Xronia: 1 kai mera: 2
Ta available ships einai [1, 3]
Den phra contract gia to available ship 3 
Xronia: 1, sunolo apo steps: 7 
To synoliko reward 

2021-11-17 13:06:40.633198: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../results/models/baseline/assets
INFO:tensorflow:Assets written to: ../results/models/policynet/assets


In [12]:
# run this cell when needing to reload the classes CarbonEnv, BaselineNet, PolicyNet

del sys.modules['models.models']
del sys.modules['env.env']
del sys.modules['training.training_functions']
del sys.modules['utils.utils']

from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel
from training.training_functions import PolicyGradient

In [ ]:
baseline_net = BaselineNet(128,1)
policy_net = PolicyNet(128,13)

In [ ]:
envo = CarbonEnv()

In [ ]:
model = PolicyGradient(envo)

In [ ]:
year_data = model.train()

In [ ]:
rewards_array = year_data['reward']

In [ ]:
states_array = year_data['states']

In [ ]:
actions_array = year_data['action']

In [ ]:
def get_returns(rewards_array):
    T = len(rewards_array)
    discounts = np.logspace(0,T,num=T, base=0.70, endpoint=False)
    returns = np.array([np.sum(discounts[:T-t] * rewards_array[t:]) for t in range(T)])
    return returns

In [ ]:
returns = get_returns(rewards_array)

In [ ]:
def get_advantage(returns, states):
    value_array = np.array([])
    for state in states:
        value = baseline_net.forward(state).numpy()
        value_array = np.append(value_array,value)
    advantages = returns - value_array
    advantages = (advantages - np.mean(advantages)) / np.sqrt(np.sum(advantages ** 2))
    return advantages

In [ ]:
advantages = get_advantage(returns,states_array)

In [ ]:
advantages

In [ ]:
one_state = states_array[0]
one_return = returns[0]
one_advantage = advantages[0]

In [ ]:
one_action = actions_array[0]

In [ ]:
one_state, one_return, one_advantage,one_action

In [ ]:
baseline_net.update

In [ ]:
one_return

In [ ]:
baseline_net.update(one_state,one_return)

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [ ]:
def update_policy(one_state, one_action, one_advantage):
    #one_state is already a tensor
    one_action = tf.convert_to_tensor(one_action)
    one_advantage = tf.convert_to_tensor(one_advantage)
    with tf.GradientTape() as tape:
        log_prob = policy_net.action_distribution(one_state)[1].log_prob(one_action)
        loss = -tf.math.reduce_mean(log_prob * tf.cast(one_advantage, tf.float32))
    grads = tape.gradient(loss, policy_net.policy_model.trainable_weights)
    optimizer.apply_gradients(zip(grads, policy_net.policy_model.trainable_weights))

In [ ]:
a = {"f":29,'d':43}

In [ ]:
for i, key in enumerate(a):
    print(f"to numero einai {i}")
    print(f"to key einai {key}")

### Tensorflow Tip

Assign values to a tensor at specific indices

In [ ]:
input_array = np.array([2, 4, 7, 11, 3, 8, 9, 19, 11, 7])
inplace_array = np.array([10, 20])
indices_array = np.array([0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

# [[2], [6]] 
indices = tf.cast(tf.where(tf.equal(indices_array,1)),tf.int32)
print(indices)

# [0, 0, 10, 0, 0, 0, 20, 0, 0, 0]
scatter = tf.scatter_nd(indices, inplace_array, shape=tf.shape(input_array))
print(scatter)

# [1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
inverse_mask = tf.cast(tf.math.logical_not(indices_array), tf.int32)
print(inverse_mask)

# [2, 4, 0, 11, 3, 8, 0, 19, 11, 7]
input_array_zero_out = tf.multiply(inverse_mask, input_array)
print(input_array_zero_out)

# [2, 4, 10, 11, 3, 8, 20, 19, 11, 7]
output = tf.add(input_array_zero_out, tf.cast(scatter, tf.int32))
print(output)

In [ ]:
# [[2], [6]] 
indices = tf.cast(tf.where(tf.equal(indices_array,1)),tf.int32)
print(indices)

In [ ]:
# [0, 0, 10, 0, 0, 0, 20, 0, 0, 0]
scatter = tf.scatter_nd(indices, inplace_array, shape=tf.shape(input_array))
print(scatter)

In [ ]:
# [1, 1, 0, 1, 1, 1, 0, 1, 1, 1]
inverse_mask = tf.cast(tf.math.logical_not(indices_array), tf.int32)
print(inverse_mask)

In [ ]:
# [2, 4, 0, 11, 3, 8, 0, 19, 11, 7]
input_array_zero_out = tf.multiply(inverse_mask, input_array)
print(input_array_zero_out)

In [ ]:
# [2, 4, 10, 11, 3, 8, 20, 19, 11, 7]
output = tf.add(input_array_zero_out, tf.cast(scatter, tf.int32))
print(output)

In [ ]:
# encoder mlp

# see https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
# see https://www.tensorflow.org/tutorials/generative/autoencoder
# Create an Integer Categorical Feature for contract_type